In [1]:
import json
import glob
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path

In [2]:
DATA_DIR = Path("/content/")

json_paths = {
    "destinos_ids": DATA_DIR / "destinos_ids.json",
    "manta": DATA_DIR / "manta_data.json",
    "salinas": DATA_DIR / "salinas_data.json",
    "montañita": DATA_DIR / "montañita_data.json",
    "puerto_lopez": DATA_DIR / "puerto_lópez_data.json",
    "general_villamil": DATA_DIR / "general_villamil_data.json",
    "ayampe": DATA_DIR / "ayampe_data.json",
    "atacames": DATA_DIR / "atacames_data.json"
}

data_raw = {}

for key, file_path in json_paths.items():
    with open(file_path, "r", encoding="utf-8") as f:
        content = json.load(f)
        data_raw[key] = content

In [3]:
records = []

for key, content in data_raw.items():
    if "alojamientos" not in content:
        continue  # destinos_ids.json no tiene alojamientos

    destino = content["destino"]

    for aloj in content["alojamientos"]:
        records.append({
            "destino": destino,
            "title": aloj.get("title"),
            "price": aloj.get("price"),
            "rating": aloj.get("rating"),
            "dist_centro_km": aloj.get("distance"),
            "description": aloj.get("description"),
            "services":aloj.get("services", []),
            "reviews": aloj.get("reviews", []),
            "features": aloj.get("features", [])
        })

df = pd.DataFrame(records)

In [4]:
df

,destino,title,price,rating,dist_centro_km,description,services,reviews,features
0,Manta,Hotel Casa Latina,US$1.729,"8,8","a 2,3 km del centro","Hotel Casa Latina se encuentra en Manta, a 4 m...","[Traslado aeropuerto, Traslado aeropuerto, Hab...",[{'title': 'Me gustó la estadía muy cómoda y p...,Desayuno incluido
1,Manta,Hotel Boutique Casa Umiña,US$1.938,"8,6","a 2,6 km del centro",Hotel Boutique Casa Umiña se encuentra en Mant...,"[Traslado aeropuerto, Traslado aeropuerto, Hab...",[{'title': 'Uno de los mejores servicios que h...,
2,Manta,Hostal Antares,US$2.930,"8,5","a 2,4 km del centro",El Hostal Antares se encuentra en Manta y ofre...,"[Piscina al aire libre, Piscina al aire libre,...","[{'title': 'Fantástico', 'date': 'Fecha del co...",
3,Manta,Hotel La Cultura,US$1.750,"7,5","a 1,6 km del centro","Hotel La Cultura se encuentra en Manta, a 2,6 ...","[Parking gratis, Parking gratis, Servicio de h...","[{'title': 'Ruido', 'date': 'Fecha del comenta...",
4,Manta,Apart Hotel Hamilton,US$3.276,"8,8","a 3,2 km del centro",El Apart Hotel Hamilton cuenta con piscina cub...,"[Piscina al aire libre, Piscina al aire libre,...","[{'title': 'Estancia muy linda y cómoda.', 'da...",Desayuno incluido
...,...,...,...,...,...,...,...,...,...
256,Atacames,Hotel Villa Turquesa,US$2.040,"9,0",a 5 km de Atacames,"Hotel Villa Turquesa se encuentra en Tonsupa, ...","[Piscina al aire libre, Piscina al aire libre,...","[{'title': 'Hospedaje limpio y tranquilo', 'da...",
257,Atacames,Playa Azul,US$3.150,"3,0","a 6,8 km de Atacames",Playa Azul es un alojamiento con terraza que s...,"[Piscina, Piscina, Parking gratis, Parking gra...",[{'title': 'Se debería seleccionar a quien se ...,
258,Atacames,"Hermoso departamento, excelente vista al mar",US$5.454,"9,3","a 6,5 km de Atacames","Hermoso departamento, excelente vista al mar e...","[Piscina al aire libre, Piscina al aire libre,...","[{'title': 'Excepcional', 'date': 'Fecha del c...",
259,Atacames,Hotel Zulema Inn,US$980,"8,5","a 24,6 km de Atacames",Hotel Zulema Inn está en Esmeraldas y tiene sa...,"[Parking gratis, Parking gratis, Servicio de h...","[{'title': 'habitacion confortable', 'date': '...",


In [5]:
df["price"] = (
    df["price"]
    .str.replace("US$", "", regex=False)
    .str.replace(".", "", regex=False)
    .str.replace(",", ".", regex=False)
    .astype(float)
)

df["rating"] = (
    df["rating"]
    .replace({"Sin puntuación": np.nan})
    .str.replace(",", ".", regex=False)
    .astype(float)
)

df["dist_centro_km"] = (
    df["dist_centro_km"]
    .str.extract(r"(\d+,\d+|\d+\.\d+|\d+)")
    [0]
    .str.replace(",", ".", regex=False)
    .astype(float)
)

In [6]:
# Umbrales basados en terciles
q1 = df["price"].quantile(0.33)
q2 = df["price"].quantile(0.66)

def categorize_price(p):
    if p <= q1: return "accesible"
    elif p <= q2: return "estandar"
    else: return "premium"

df["categoria"] = df["price"].apply(categorize_price)

In [7]:
def has_pool(services):
    return any("piscina" in s.lower() for s in services)

def has_breakfast(features, services):
    text = (features or "") + " " + " ".join(services)
    return "desayuno" in text.lower()

def has_beachfront(services):
    return any("frente a la playa" in s.lower() for s in services)

df["pool"] = df["services"].apply(has_pool)
df["breakfast"] = df.apply(lambda r: has_breakfast(r["features"], r["services"]), axis=1)
df["beachfront"] = df["services"].apply(has_beachfront)

**Capacidad Hospitalaria (CH)**
Esta definida por los siguientes criterios:

1.   Total de Alojamientos (TA)
2.   Distribucion por categoria (DC)
3.   Proximidad media al centro (PC)
4.   Proporcion de alojamientos con servicios criticos (SC)

La ponderación de este criterio se dará de la siguiente manera:

CH = 0.4TA + 0.25DC + 0.25PC + 0.1SC





In [8]:
#1. Total Alojamientos (TA)
TA = df.groupby('destino')["title"].count().rename("TA")
TA

,TA
destino,
Atacames,25
Ayampe,63
General Villamil,27
Manta,40
Montañita,47
Puerto López,30
Salinas,29


Descripción de qué tan equitativa es la distribución de los alojamientos por cada categoría de precios. En otras palabras, un valor cercano a 1 indica una distribución más balanceada, la misma cantidad de alojamientos entre cada categoría. Adicional, se detalla la proporción de alojamientos por categoría para cada destino:

In [9]:
#2. Distribucion por categoria (DC)
cat_counts = df.groupby(["destino", "categoria"])["title"].count().unstack().fillna(0)
DC = 1 - (cat_counts.max(axis=1) - cat_counts.min(axis=1)) / cat_counts.sum(axis=1)
DC.name = "DC"
proportion_by_category = cat_counts.div(cat_counts.sum(axis=1), axis=0)
combined_cats = pd.concat([proportion_by_category, DC], axis=1)
display(combined_cats)

,accesible,estandar,premium,DC
destino,,,,
Atacames,0.120000,0.360000,0.520000,0.600000
Ayampe,0.412698,0.301587,0.285714,0.873016
General Villamil,0.222222,0.370370,0.407407,0.814815
Manta,0.350000,0.400000,0.250000,0.850000
Montañita,0.382979,0.276596,0.340426,0.893617
Puerto López,0.433333,0.333333,0.233333,0.800000
Salinas,0.206897,0.310345,0.482759,0.724138


Aquí tienes una tabla que muestra la distancia media al centro (`dist_mean`) y la proximidad media al centro (`PC`) para cada destino:

In [10]:
#3. Proximidad media al centro (PC)
dist_mean = df.groupby("destino")["dist_centro_km"].mean()
PC = 1 - (dist_mean - dist_mean.min()) / (dist_mean.max() - dist_mean.min())
PC.name = "PC"
combined_distances = pd.concat([dist_mean, PC], axis=1)
display(combined_distances)

,dist_centro_km,PC
destino,,
Atacames,6.100000,0.949122
Ayampe,23.144444,0.675779
General Villamil,48.740741,0.265289
Manta,2.927500,1.000000
Montañita,65.282979,0.000000
Puerto López,51.920000,0.214303
Salinas,39.337931,0.416083


In [11]:
#4. Proporcion de alojamientos con servicios criticos (SC)
SC = (
    (df.groupby("destino")["pool"].mean() +
     df.groupby("destino")["breakfast"].mean() +
     df.groupby("destino")["beachfront"].mean()) / 3
).rename("SC")
SC

,SC
destino,
Atacames,0.586667
Ayampe,0.333333
General Villamil,0.506173
Manta,0.258333
Montañita,0.333333
Puerto López,0.344444
Salinas,0.413793


In [12]:
# Normalización 0–1
CH_df = pd.concat([TA, DC, PC, SC], axis=1)
CH_norm = (CH_df - CH_df.min()) / (CH_df.max() - CH_df.min())

CH_df["CH"] = (
    0.40 * CH_norm["TA"] +
    0.25 * CH_norm["DC"] +
    0.25 * CH_norm["PC"] +
    0.1 * CH_norm["SC"]
)
CH_df

,TA,DC,PC,SC,CH
destino,,,,,
Atacames,25,0.600000,0.949122,0.586667,0.337281
Ayampe,63,0.873016,0.675779,0.333333,0.824247
General Villamil,27,0.814815,0.265289,0.506173,0.345763
Manta,40,0.850000,1.000000,0.258333,0.620757
Montañita,47,0.893617,0.000000,0.333333,0.504422
Puerto López,30,0.800000,0.214303,0.344444,0.302724
Salinas,29,0.724138,0.416083,0.413793,0.299171


**Nivel de Hospitalidad (NH)**
Esta definida con base a las ponderaciones de los siguientes criterios:
1.   Rating promedio del destino (RP)
2.   Mediana del rating (MR)
3.   Analisis de sentimiento (HF)
4.   Hospitality Experience Score (HES)

La ponderación de este criterio se dará de la siguiente forma:

CH = 0.25HF + 0.2HES + 0.2MR + 0.35RP

In [44]:
# Extraer reseñas a un DF
rev_rows = []
for _, row in df.iterrows():
    for r in row["reviews"]:
        text = (r.get("positive_feedback", "") or "") + " " + (r.get("negative_feedback", "") or "")
        rev_rows.append({"destino": row["destino"], "review": text})

reviews["review"] = reviews["review"].fillna("")

reviews = pd.DataFrame(rev_rows)
reviews

,destino,review
0,Manta,La atención es personalizada y muy atentos Hub...
1,Manta,El protocolo de la recepción e información. Co...
2,Manta,Excelente Nada
3,Manta,El desayuno excelente
4,Manta,Cerca a todo La habitación en tercer piso alto...
...,...,...
4364,Atacames,Ubicación Precio
4365,Atacames,"LITERALLY EVERY THING!!\nGreat WiFi, clean roo..."
4366,Atacames,"The staff was very helpful and cooperative, an..."
4367,Atacames,"La gentillesse du patron, sa discrétion, ses c..."


In [14]:
#1. Rating promedio del destino (RP) y 2. Mediana del rating (MR)
RP = df.groupby("destino")["rating"].mean().rename("RP")
MR = df.groupby("destino")["rating"].median().rename("MR")
Rating = pd.concat([RP, MR], axis=1)
Rating

,RP,MR
destino,,
Atacames,8.380000,8.70
Ayampe,8.407547,8.70
General Villamil,8.526923,8.75
Manta,7.780000,8.50
Montañita,8.323684,8.55
Puerto López,8.422222,8.80
Salinas,8.728571,8.80


In [45]:
#3. Analisis de sentimiento (HF)
from transformers import pipeline
import numpy as np

sentiment_pipe = pipeline(
    "sentiment-analysis",
    model="nlptown/bert-base-multilingual-uncased-sentiment"
)

def analizar_sentimientos(textos, batch_size=32):
    """
    Realiza análisis de sentimientos en español usando NLPTown.
    Procesa en lotes para no tardar demasiado.
    """
    resultados = []
    for i in range(0, len(textos), batch_size):
        batch = textos[i:i + batch_size]
        batch_results = sentiment_pipe(batch, truncation=True)
        resultados.extend(batch_results)
    return resultados

# Ejecutar el análisis con todas las reseñas
result_sentiment_analysis = analizar_sentimientos(reviews["review"].tolist())

def label_to_stars(label: str) -> float:
    """
    Convierte etiquetas tipo '4 stars' o '1 star' a float (1.0–5.0).
    """
    try:
        return float(label.split()[0])
    except Exception:
        return 3.0  # neutro si algo raro pasa

# Añadimos la columna HF (score de 1 a 5) a cada reseña
reviews["HF"] = [label_to_stars(r["label"]) for r in result_sentiment_analysis]

# Agregamos HF a nivel destino (criterio para NH)
HF_score = reviews.groupby("destino")["HF"].mean().rename("HF")

Device set to use cpu


In [46]:
HF_score

,HF
destino,
Atacames,3.717742
Ayampe,3.869818
General Villamil,3.716895
Manta,3.693405
Montañita,3.850602
Puerto López,3.973941
Salinas,3.948276


In [47]:
#4. Hospitality Experience Score (HES)
from sentence_transformers import SentenceTransformer
from sklearn.cluster import KMeans
import numpy as np

# 1) Cargar modelo de embeddings multilingüe
emb_model = SentenceTransformer("sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2")

# 2) Calcular embeddings de cada reseña
#    (si hay muchas reseñas, esto tarda un poco, pero suele ser razonable)
reviews["embedding"] = reviews["review"].apply(lambda txt: emb_model.encode(txt if isinstance(txt, str) else ""))

# 3) Preparar matriz de características para clustering
X = np.vstack(reviews["embedding"].values)

# 4) Clustering temático (ajusta n_clusters si quieres más/menos granularidad)
n_clusters = 8  # por ejemplo
kmeans = KMeans(n_clusters=n_clusters, random_state=42, n_init="auto")
reviews["cluster"] = kmeans.fit_predict(X)

# 5) Calcular promedio de HF por destino y cluster (sentimiento por tema)
cluster_sent = reviews.groupby(["destino", "cluster"])["HF"].mean()

# 6) Hospitality Experience Score:
#    promedio del sentimiento por cluster para cada destino
HES = cluster_sent.groupby("destino").mean().rename("HES")

In [48]:
#Nivel de Hospitalidad (NH)
NH_df = pd.concat([RP, MR, HF_score, HES], axis=1).fillna(0)

# Normalización min–max
NH_norm = (NH_df - NH_df.min()) / (NH_df.max() - NH_df.min())

# Fórmula que definiste para NH:
# NH = 0.30 RP + 0.15 MR + 0.30 HF + 0.25 HES
NH_df["NH"] = (
    0.30 * NH_norm["RP"] +
    0.15 * NH_norm["MR"] +
    0.30 * NH_norm["HF"] +
    0.25 * NH_norm["HES"]
)
NH_df

,RP,MR,HF,HES,NH
destino,,,,,
Atacames,8.380000,8.70,3.717742,3.795252,0.315785
Ayampe,8.407547,8.70,3.869818,3.856617,0.574705
General Villamil,8.526923,8.75,3.716895,3.874854,0.499955
Manta,7.780000,8.50,3.693405,3.822397,0.038742
Montañita,8.323684,8.55,3.850602,3.844756,0.435706
Puerto López,8.422222,8.80,3.973941,3.970418,0.903112
Salinas,8.728571,8.80,3.948276,3.912472,0.889851


**Relacion Calidad-Precio (RCP)**
Esta definida con base a las ponderaciones de los siguientes criterios:
1.   Precio promedio por categoria (PPC)
2.   Precio vs. Rating (PVR)
3.   Dispersion de precios (DP)

La ponderación de este criterio se dará de la siguiente manera:

CH = 0.4PPC + 0.35PVR + 0.25DP  

In [28]:
#1. Precio promedio por categoria (PPC)
PPC = df.groupby(["destino", "categoria"])["price"].mean().unstack().fillna(0)
PPC = PPC.mean(axis=1).rename("PPC")

In [29]:
#2. Precio vs. Rating (PVR)
df["PVR"] = df["rating"] / df["price"]
PVR = df.groupby("destino")["PVR"].mean().rename("PVR")

In [30]:
#3. Dispersion de precios (DP)
DP = df.groupby("destino")["price"].std().fillna(0).rename("DP")

In [32]:
combined_quality_price = pd.concat([PPC, PVR, DP], axis=1)
display(combined_quality_price)

,PPC,PVR,DP
destino,,,
Atacames,3162.113960,0.002777,1862.517890
Ayampe,3097.562678,0.004656,2003.125648
General Villamil,3505.533333,0.003201,2821.475713
Manta,3245.262500,0.003759,1848.042941
Montañita,3176.595264,0.004350,2313.925011
Puerto López,3173.247253,0.004648,1862.037430
Salinas,3891.534392,0.003456,3573.047252


### Explicación y mejora de la presentación de los resultados:

Aquí se muestra una tabla combinada de los indicadores relacionados con la calidad-precio para cada destino:

**1. Precio promedio por categoría (PPC)**
*   **Descripción:** Este métrica representa el precio promedio de los alojamientos en cada destino, considerando la distribución de precios por categoría (accesible, estándar, premium). Un valor menor indica que, en promedio, los alojamientos en el destino tienen precios más bajos.
*   **Indicaciones:** Es útil para entender el nivel de costo general de un destino. Por ejemplo, un `PPC` alto en Salinas podría indicar que es un destino con alojamientos más caros en general.

**2. Precio vs. Rating (PVR)**
*   **Descripción Faltante:** El `PVR` mide la relación entre la calificación (rating) de un alojamiento y su precio. Se calcula como la calificación dividida por el precio (`rating / price`).
*   **Indicaciones:** Un valor `PVR` **más alto** sugiere que los alojamientos en ese destino ofrecen una **mejor percepción de valor**, es decir, obtienen calificaciones más altas en relación con su precio. Por el contrario, un `PVR` bajo podría indicar que los precios son altos en comparación con las calificaciones recibidas.
*   **Mejor forma de mostrarlo:** Además de la tabla, un gráfico de barras para cada destino que muestre el `PVR` puede hacer que las comparaciones sean más intuitivas. También se podría considerar un gráfico de dispersión con `PPC` en un eje y `PVR` en otro para ver patrones entre el costo general y la percepción de valor.

**3. Dispersión de precios (DP)**
*   **Descripción Faltante:** La `DP` representa la desviación estándar de los precios de los alojamientos en cada destino. Este valor indica la variabilidad o el rango de precios dentro del destino.
*   **Indicaciones:** Un `DP` **alto** sugiere que hay una **gran variedad de precios** en el destino, lo que implica opciones para diferentes presupuestos. Un `DP` bajo, en cambio, indica que los precios de los alojamientos son más homogéneos y menos variados.
*   **Mejor forma de mostrarlo:** Un gráfico de barras para la `DP` podría visualizar fácilmente qué destinos tienen una mayor o menor diversidad de precios. Combinarlo con el `PPC` en un mismo gráfico (por ejemplo, con un eje secundario) también podría ofrecer una visión completa de la estructura de precios de cada destino.

In [33]:
# Fórmula RCP
RCP_df = pd.concat([PPC, PVR, DP], axis=1)
RCP_norm = (RCP_df - RCP_df.min()) / (RCP_df.max() - RCP_df.min())

RCP_df["RCP"] = (
    0.40 * RCP_norm["PPC"] +
    0.40 * RCP_norm["PVR"] +
    0.20 * RCP_norm["DP"]
)

In [49]:
final = pd.concat([CH_df["CH"], NH_df["NH"], RCP_df["RCP"]], axis=1)

final["Score_Final"] = (
    0.4 * final["CH"] +
    0.3 * final["NH"] +
    0.2 * final["RCP"]
)

ranking = final.sort_values("Score_Final", ascending=False)
ranking

,CH,NH,RCP,Score_Final
destino,,,,
Ayampe,0.824247,0.574705,0.417981,0.585706
Salinas,0.299171,0.889851,0.744639,0.535552
Puerto López,0.302724,0.903112,0.437990,0.479621
Montañita,0.504422,0.435706,0.428658,0.418212
General Villamil,0.345763,0.499955,0.408739,0.370039
Manta,0.620757,0.038742,0.283548,0.316635
Atacames,0.337281,0.315785,0.034199,0.236487
